In [0]:
cd /content/drive/My Drive/ITSP-2020

/content/drive/.shortcut-targets-by-id/103/ITSP-2020


In [0]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.2.0


In [0]:
!pip install tensorflow==1.10

     |████████████████████████████████| 58.4MB 76kB/s 
     |████████████████████████████████| 12.2MB 45.0MB/s 
     |████████████████████████████████| 3.3MB 34.8MB/s 
     |████████████████████████████████| 573kB 50.9MB/s 
ERROR: xarray 0.15.1 has requirement numpy>=1.15, but you'll have numpy 1.14.5 which is incompatible.
ERROR: xarray 0.15.1 has requirement setuptools>=41.2, but you'll have setuptools 39.1.0 which is incompatible.
ERROR: umap-learn 0.4.3 has requirement numpy>=1.17, but you'll have numpy 1.14.5 which is incompatible.
ERROR: tifffile 2020.5.25 has requirement numpy>=1.15.1, but you'll have numpy 1.14.5 which is incompatible.
ERROR: spacy 2.2.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.5 which is incompatible.
ERROR: plotnine 0.6.0 has requirement numpy>=1.16.0, but you'll have numpy 1.14.5 which is incompatible.
ERROR: numba 0.48.0 has requirement numpy>=1.15, but you'll have numpy 1.14.5 which is incompatible.
ERROR: imgaug 0.2.9 has requirement nump

In [0]:
import os
from Latex.Latex import Latex
import numpy as np
from os import listdir
from skimage import io
import matplotlib.pyplot as plt
from IPython.display import display, Math
import re
import json
import tensorflow as tf


In [0]:
mean_train = np.load("train_images_mean.npy")
std_train = np.load("train_images_std.npy")

In [0]:
tf.reset_default_graph()


In [0]:
model = Latex("model", mean_train, std_train, plotting=False)

In [0]:
files = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser("formulas/train")) for f in fn]

In [0]:
len(files)

16879

In [0]:
formula = io.imread(files[0])

In [0]:
formula = io.imread(files[0])
pos = files[0].rfind("/")
filename = files[0][pos+1:]
height, width = formula.shape
correct = model.filename2formula(filename)
oseq = model.filename2seq(filename)


['#int', ' ', '#frac', ' ', '{', '#tanx', '}', '{', '4', '2', '2', '}', 'x', '^', '2', '-', '5', '1', '7', '-', '#cosx', ' ', '+', '5', '6', '5', 'd', 'x', '.', 'j', 'p']


In [0]:
filename

'#int #frac {#tanx}{422}x^2-517-#cosx +565dx.jpg'

In [0]:
correct

'\\ int \\ frac {tanx}{422}x^2-517-cosx +565dx.jp'

In [0]:
ltokens = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-', '+', '=', '#leq', '#neq', '#geq', '#alpha',
                            '#beta', '#lambda', '#lt', '#gt', 'x', 'y', '^', '#frac', '{', '}' ,' ','#int','d','#sinx','#cosx','#tanx']

In [0]:
nlabels = 28
nof_sequences = len(files)
nclasses = nlabels+4+2+1 # 1 for pad and 4 for relative pos, 2 for abs pos and shift from last and width
noclasses = len(ltokens)
isequence = np.zeros((nof_sequences,35,nclasses))
isequence[:,:,-1] = 1 # default all pad
osequence = (noclasses-1)*np.ones((nof_sequences,30), dtype=int)
osequence[:,0] = noclasses # all <GO> in beginning
for i in range(nof_sequences):
    if i % 10 == 0:
        print("Start i: %d" % i)
    formula = io.imread(files[i])
    pos = files[i].rfind("/")
    filename = files[i][pos+1:]
    height, width = formula.shape
    correct = model.filename2formula(filename)
    oseq = model.filename2seq(filename)
    if(len(oseq)>=30):
      continue  
    osequence[i,1:len(oseq)+1] = oseq
    latex = model.predict(formula)
    
    last_xmax = 0
    last_ymin = latex['data'][0]['ymin']
    step_c = -1
    for step in latex['data']:
        step_c += 1
        isequence[i][step_c][:nlabels] = step['probs']
        isequence[i][step_c][-1] = 0 # remove pad
        isequence[i][step_c][-7] = step['xmin']/width
        isequence[i][step_c][-6] = step['ymin']/height
        isequence[i][step_c][-5] = (step['xmin']-last_xmax)/10
        last_xmax = step['xmax']
        isequence[i][step_c][-4] = (step['xmax']-step['xmin'])/48
        isequence[i][step_c][-3] = (step['ymin']-last_ymin)/10
        isequence[i][step_c][-2] = (step['ymax']-step['ymin'])/48
        last_ymin = step['ymin']

Streaming output truncated to the last 5000 lines.
['#int', ' ', '#frac', ' ', '{', '#tanx', '}', '{', '2', '2', '9', '}', 'x', '^', '4', '-', '5', '0', '9', '-', '#tanx', ' ', '+', '2', '8', '3', 'd', 'x', '.', 'j', 'p']
['#int', ' ', '#frac', ' ', '{', '9', '2', '7', '}', '{', '5', '6', '8', '}', 'x', '^', '2', '-', '#sinx', '-', '#sinx', ' ', '+', '4', '6', '7', 'd', 'x', '.', 'j', 'p']
['#int', ' ', '#frac', ' ', '{', '#sinx', '}', '{', '2', '1', '7', '}', 'x', '^', '3', '-', '#tanx', '-', '#cosx', ' ', '+', '9', '1', '5', 'd', 'x', '.', 'j', 'p']
['#int', ' ', '#frac', ' ', '{', '#tanx', '}', '{', '7', '2', '}', 'x', '^', '3', '+', '2', '8', '+', '#sinx', ' ', '+', '9', '5', '8', 'd', 'x', '.', 'j', 'p']
['#int', ' ', '#frac', ' ', '{', '#sinx', '}', '{', '7', '9', '3', '}', 'x', '^', '3', '-', '#tanx', '-', '#sinx', ' ', '+', '4', '0', '0', 'd', 'x', '.', 'j', 'p']
['#int', ' ', '#frac', ' ', '{', '#sinx', '}', '{', '6', '9', '8', '}', 'x', '^', '2', '-', '#tanx', '-', '#cosx', '

/usr/local/lib/python3.6/dist-packages/scipy/ndimage/measurements.py:1359: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]


Streaming output truncated to the last 5000 lines.
['#int', ' ', '#frac', ' ', '{', '7', '0', '1', '}', '{', '2', '3', '4', '}', 'x', '^', '3', '-', '#sinx', '-', '#tanx', ' ', '+', '3', '7', '2', 'd', 'x', '.', 'j', 'p']
['#int', ' ', '#frac', ' ', '{', '2', '7', '4', '}', '{', '5', '0', '7', '}', 'x', '^', '2', '-', '#tanx', '-', '#cosx', ' ', '+', '5', '9', '4', 'd', 'x', '.', 'j', 'p']
['#int', ' ', '#frac', ' ', '{', '#sinx', '}', '{', '1', '8', '2', '}', 'x', '^', '3', '-', '9', '3', '3', '-', '#tanx', ' ', '+', '8', '7', '2', 'd', 'x', '.', 'j', 'p']
['#int', ' ', '#frac', ' ', '{', '7', '9', '0', '}', '{', '7', '5', '2', '}', 'x', '^', '2', '+', '#tanx', '+', '#sinx', ' ', '+', '1', '4', '6', 'd', 'x', '.', 'j', 'p']
['#int', ' ', '#frac', ' ', '{', '8', '0', '1', '}', '{', '6', '0', '3', '}', 'x', '^', '4', '-', '4', '1', '6', '-', '#cosx', ' ', '+', '7', '1', '4', 'd', 'x', '.', 'j', 'p']
['#int', ' ', '#frac', ' ', '{', '#sinx', '}', '{', '2', '9', '3', '}', 'x', '^', '4', '

In [0]:
np.save("iseq_n", isequence)
np.save("oseq_n", osequence)
with open('files.json', 'w') as f:
    f.write(json.dumps(files[:nof_sequences]))